### Loading jsonl file

In [ ]:
## install requred libraries
!pip install fastparquet

In [8]:
## Libraries
import glob

# - to extract gz files
import gzip
import json

# - to convert json to dataframe
import pandas as pd

In [52]:
# Load file
filename = "../data/raw/2021-us-patent.jsonl.gz"

## Open the gzip file and read the contents, apply decode to convert from bytes to string.
with gzip.open(filename, 'rb') as f:
    file_content = f.read().decode('ascii',  errors='ignore')

f.close()

In [53]:
content = file_content.splitlines()[9577]

In [54]:
content

'{"lens_id":"103-612-129-312-964","jurisdiction":"US","doc_number":"10922450","kind":"B2","date_published":"2021-02-16","doc_key":"US_10922450_B2_20210216","docdb_id":545960357,"lang":"en","biblio":{"publication_reference":{"jurisdiction":"US","doc_number":"10922450","kind":"B2","date":"2021-02-16"},"application_reference":{"jurisdiction":"US","doc_number":"201514915990","kind":"A","date":"2015-01-13"},"priority_claims":{"claims":[{"jurisdiction":"US","doc_number":"201514915990","kind":"A","date":"2015-01-13","sequence":1},{"jurisdiction":"US","doc_number":"201462009640","kind":"P","date":"2014-06-09","sequence":2},{"jurisdiction":"US","doc_number":"2015011105","kind":"W","date":"2015-01-13","sequence":3}],"earliest_claim":{"date":"2014-06-09"}},"invention_title":[{"text":"Associating computer-executable objects with timber frames within an architectural design environment","lang":"en"}],"parties":{"examiners":{"primary_examiner":{"department":"2127","extracted_name":{"value":"Kamini S

In [34]:
patent = json.loads(content)

JSONDecodeError: Unterminated string starting at: line 1 column 30218 (char 30217)

In [16]:
classifications_cpc = patent['biblio'].get('classifications_cpc')

In [34]:
dict = patent['biblio']['parties']['applicants'][0]
dict.get('residence')

In [38]:
dict.get('extracted_name', 'other')

{'value': 'HISEP TECH LTD'}

In [8]:
patents_data = []
patents_classifications = []
patents_applicants = []
patents_inventors = []

In [ ]:
for content in file_content.splitlines():
    ## load content as a patent
    patent = json.loads(content)
    data = {
        'lens_id': patent['lens_id'],
        'jurisdiction': patent['jurisdiction'],
        'patent_id': patent['doc_key'],
        'date_published': patent['date_published'],
        'title': patent['biblio']['invention_title'][0]['text'],
        'abstract': patent['abstract'][0]['text']
    }
    patents_data.append(data)

    for applicant in patent['biblio']['parties']['applicants']:
        app_data = {
            'lens_id': patent['lens_id'],
            'patent_id': patent['doc_key'],
            'residence': applicant.get('residence', 'NA'),
            'name': applicant['extracted_name']['value']
        }
        patents_applicants.append(app_data)
    
    for inventor in patent['biblio']['parties']['inventors']:
        inv_data = {
            'lens_id': patent['lens_id'],
            'patent_id': patent['doc_key'],
            'residence': inventor.get('residence', 'NA'),
            'name': inventor['extracted_name']['value']
        }
        patents_inventors.append(inv_data)

    for classification in patent['biblio']['classifications_cpc']['classifications']:
        class_data = {
            'lens_id': patent['lens_id'],
            'patent_id': patent['doc_key'],
            'classification': classification['symbol']
        }
        patents_classifications.append(class_data)

del(content)    ## clear variable from memory

In [11]:
df = pd.DataFrame(patents_data)

In [14]:
df.to_parquet('test.parquet')

In [55]:
def process_file(file_content):

    for content in file_content.splitlines():
        ## load content as a patent
        patent = json.loads(content)

        data = {
            'lens_id': patent['lens_id'],
            'jurisdiction': patent['jurisdiction'],
            'patent_id': patent['doc_key'],
            'date_published': patent['date_published'],
            'title': patent['biblio']['invention_title'][0]['text'],
            'abstract': patent['abstract'][0]['text']
        }
        patents_data.append(data)

        for applicant in patent['biblio']['parties']['applicants']:
            app_data = {
                'lens_id': patent['lens_id'],
                'patent_id': patent['doc_key'],
                'residence': applicant.get('residence', 'NA'),
                'name': applicant['extracted_name']['value']
            }
            patents_applicants.append(app_data)
        
        for inventor in patent['biblio']['parties']['inventors']:
            inv_data = {
                'lens_id': patent['lens_id'],
                'patent_id': patent['doc_key'],
                'residence': inventor.get('residence', 'NA'),
                'name': inventor['extracted_name']['value']
            }
            patents_inventors.append(inv_data)

        #print('process classifications')
        classifications_cpc = patent['biblio'].get('classifications_cpc')
        if classifications_cpc is not None:
            for classification in classifications_cpc['classifications']:
                #print('process classification')
                class_data = {
                    'lens_id': patent['lens_id'],
                    'patent_id': patent['doc_key'],
                    'classification': classification['symbol']
                }
                patents_classifications.append(class_data)

    del(content)    ## clear variable from memory
   

In [56]:
def process_gzip(filename):
    ## Open the gzip file and read the contents, apply decode to convert from bytes to string.
    with gzip.open(filename, 'rb') as f:
        file_content = f.read().decode('ascii',  errors='ignore')       ##handling special characters: ignore, skip characters.
        process_file(file_content)
    f.close()

In [44]:
path = r'..\data\raw\*.gz'
files = glob.glob(path)

## init dicts:
patents_data = []
patents_classifications = []
patents_applicants = []
patents_inventors = []


for f in files:
    print("processing: ", f)
    process_gzip(f)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

JSONDecodeError: Unterminated string starting at: line 1 column 30218 (char 30217)

In [38]:
# Load file
filename = "../data/raw/poor_text"

## Open the gzip file and read the contents, apply decode to convert from bytes to string.
with open(filename, 'rb') as f:
    file_content = f.read().decode('utf8')

f.close()

In [40]:
test = json.loads(file_content)

In [41]:
test

{'test': 'SIGGRAPH 99, ACM �\x85 - NEW YORK,'}